In [ ]:
import requests
import urllib.request
import time
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import warnings
from math import ceil
warnings.filterwarnings("ignore")


# open_page Function

In [ ]:
time_sleep = 2 #Sec
def open_page(url, time_sleep):
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    time.sleep(time_sleep)
    return(soup)

# ScholarshipDb.Net

In [ ]:
category = "Scholarship" 
program = "PhD" 
query = "coastal engineering"
query = input()
listed = 30
page = 1
#make url
def make_url(category, program, query, listed, page):
    query = "+".join(query.split())
    url = "https://scholarshipdb.net/scholarships/"  
    if category:
        url = url + "Category-{}/".format(category)
    if program:
        url = url + "Program-{}?".format(program)
    if query:
        url = url + "q={}&".format(query)
    if listed:
        url = url + "listed=Last-{}-days&".format(str(listed))
    if page:
        url = url + "l=&page={}".format(str(page))
    return url
url = make_url (category, program, query, listed, page)
#find the page
soup = open_page(url, time_sleep)
pages = list(range(1, ceil(int(soup.select("h1.title")[0].text.split()[0])/10)+1))

df = []
for page in pages:
    url = make_url (category, program, query, listed, page)
    soup = open_page(url, time_sleep)
    section = soup.select("ul.list-unstyled")[4].select("li")
    job_hyperlink = []
    for i in range(len(section)):
        try:
            job_hyperlink.append("https://scholarshipdb.net/" + section[i].find("div").select("a")[-1]["href"])
        except AttributeError:
            pass
    df.extend(job_hyperlink)
for url in df:
    soup = open_page(url, time_sleep)
    title = soup.select("div.position-details")[0].find("div").text.lstrip().rstrip()
    try:
        Deadline = soup.find("span", text="Deadline: ").next_sibling
    except AttributeError:
        Deadline = None
    try:
        Updated = soup.find("span", text="Updated: ").next_sibling.text
    except AttributeError:
        Updated = None    
    try:
        Location = soup.find("span", text="Location: ").next_sibling.text
    except AttributeError:
        Location = None
    try:    
        University = soup.select("div.position-details")[0].find("h2").find("a").text
    except AttributeError:
        University = None    
    print(title)

In [ ]:
columns = ["job_title", "job_location", "job_country", "job_time", "job_introduction", "job_hyperlink"]

In [ ]:
df = []
for page in pages:
    url = make_url (category, program, query, listed, page)
    soup = open_page(url)
    time.sleep(3)
    d = extract_job_defintion_from_result(soup)
    d = dict(tuple(list(enumerate(d))))
    df2 = pd.DataFrame(d) 
    df.append(df2)   
df = pd.concat(df, ignore_index=True)
df.columns = columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Find A PhD

In [ ]:
Funding = "non-eu-students" 
Keyword = input()
page = 1
def make_url (Keyword, Funding, page):
    Keyword = "+".join(Keyword.split())
    url = "https://www.findaphd.com/phds/"   
    if Funding:
        url = url + "{}".format(Funding)
    if Keyword:
        url = url + "/?01w0&Keywords={}".format(Keyword)
    if page:
        url = url + "&PG={}".format(str(page))
    return url
url = make_url (Keyword, Funding, page)
soup = open_page(url, time_sleep)
pages = list(range(1, ceil(int(soup.select("div.land")[1].find("h4").text.split()[2])/10)+1))

df = []
for page in pages:
    url = make_url (Keyword, Funding, page)
    soup = open_page(url, time_sleep):

    links = soup.select("a.ctaButton.details")
    job_hyperlink = ["https://www.findaphd.com" + link['href'] for link in links]
    df.extend(job_hyperlink)
for url in df:
    soup = open_page(url, time_sleep):
    title = soup.select("h1")[0].text
    try:
        university = soup.select("h2.instLink")[0].text
    except:
        university = None
    try:
        department = soup.select("h2.deptLink")[0].text
    except:
        department = None
    try:
        country = soup.select("h2.ctryLink")[0].text
    except:
        country = None        
    try:
        supervisor = soup.select("li.toItem.col-sm-12.col-xs-12")[0].find("a", attrs={'class':'emailLink'}).text
    except:
        supervisor = None
    try:
        body = soup.select("div.wrapChildUrls")[0].text
    except:
        body = None
    print(title)
    print(university)
    print(department)
    print(country)
    print(supervisor)
print("Finished")
print(100*".")

# EURAXESS 

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
Keyword = input()
page = 1
def make_url (Keyword, page):    
    Keyword = "+".join(Keyword.split())
    url = "https://euraxess.ec.europa.eu/jobs/search?keywords="   
    if Keyword:
        url = url + "{}".format(Keyword)
    if page:
        url = url + "&page={}".format(str(page))
    return url
url = make_url (Keyword, page)
soup = open_page(url, time_sleep):
pages = list(range(1, int(ceil(int(soup.select("h2.text-center")[0].text.split()[-1].replace("(", "").replace(")", "")))/10)+1))
print(pages)
df = []
for page in pages:
    url = make_url (Keyword, page)
    soup = open_page(url, time_sleep):
    links = soup.select("a.btn.btn-link.action")[:-1]
    job_hyperlink = ["https://euraxess.ec.europa.eu/" + link['href'] for link in links]
    df.extend(job_hyperlink)
    
for url in df:
    soup = open_page(url, time_sleep):
    title= soup.select("h1.head-title")[0].text
    institute = soup.select("div.col-xs-12.col-sm-7.value.field-company-institute.inline")[0].text
    deadline = soup.select("div.col-xs-12.col-sm-7.value.field-application-deadline.inline")[0].text
    country = soup.select("div.col-xs-12.col-sm-7.value.field-country.inline")[0].text
    body = soup.select("div.longtext.field-body")[0].text
    print(title, deadline)
